In [1]:
import PyPDF2
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter

In [2]:
def extract_event_info(event):
    lines = event.strip().split('\n')
    event_number = int(lines[0])
    picked_arrival = re.search(r'Picked arrival: (.+)', event).group(1)
    tag = re.search(r'Tag : (.+)', event).group(1)
    arrival_type = re.search(r'Arrival type : (.+)', event).group(1)
    iris_event_id = re.search(r'IRIS Event ID : (\d+)', event).group(1)

    return [event_number, picked_arrival, tag, arrival_type, iris_event_id]

pdf_path = 'Pipatprathanporn+2021-SM1.pdf'

info = []
# Ouvrir le fichier PDF en mode binaire
with open(pdf_path, 'rb') as file:
    # Initialiser le lecteur PDF
    pdf_reader = PyPDF2.PdfReader(file)

    # Parcourir les pages du PDF
    for page_num in range(1, len(pdf_reader.pages)):  # Commencer à la deuxième page
        # Obtenir le contenu de la page
        page = pdf_reader.pages[page_num]
        page_content = page.extract_text()

        # Extraire la valeur de "Picked arrival"
        info_event = extract_event_info(page_content)
        info.append(info_event)

info = np.array(info)
df = pd.DataFrame(info, columns=['Event number', 'Picked arrival', 'Tag', 'Arrival type', 'IRIS Event ID'])
df.head()

,Event number,Picked arrival,Tag,Arrival type,IRIS Event ID
0,1,2018-11-18 20:32:00.000,DET,body,10972756
1,2,2018-11-30 17:42:00.000,DET,body,10976411
2,3,2018-12-05 04:26:35.000,DET,body,10980271
3,4,2018-12-11 02:38:40.000,DET,body,10983619
4,5,2018-12-23 23:14:45.000,DET,body,10988734


In [3]:
df['Picked arrival'] = pd.to_datetime(df['Picked arrival'], format='mixed', errors='coerce')

In [4]:
df['file'] = ['vide' for _ in range(len(df))]
df['file_start'] = ['vide' for _ in range(len(df))]

In [5]:
for i in range(len(df)):
    if i%10 == 0:
        print(i, '/', len(df))
    event_time = df.loc[i,'Picked arrival']
    for dossier_actuel, sous_dossiers, fichiers in os.walk('data\DonneesB23'):
        for fichier in fichiers:
            if fichier[0] == '2' :
                fichier_time = datetime.strptime(fichier[:19], '%Y-%m-%dT%H_%M_%S')
                fichier_time = fichier_time.replace(microsecond=0)
                if event_time >= fichier_time:
                        fs = 40
                        path = os.path.join(dossier_actuel, fichier)
                        y = np.fromfile(path, dtype=np.int32)
                        duree_sigal_s = int(len(y)/fs)
                        if (event_time - fichier_time).total_seconds() < duree_sigal_s:
                            if df.loc[i,'file'] != 'vide':
                                print("Attention, deux fichiers pour l'événement {} !".format(i))
                            df.loc[i,'file'] = path
                            df.loc[i,'file_start'] = fichier_time
                            break
                        

0 / 213
10 / 213
20 / 213
30 / 213
40 / 213
50 / 213
60 / 213
70 / 213
80 / 213
90 / 213
100 / 213
110 / 213
120 / 213
130 / 213
140 / 213
150 / 213
160 / 213
170 / 213
180 / 213
190 / 213
200 / 213
210 / 213


In [6]:
fichiers = df['file'].to_numpy()

In [7]:
(fichiers == 'vide').sum()/len(fichiers)


0.0

In [8]:
df.to_csv('info_events.csv', index=False)